# Lesson 3 Pick and Place

Welcome to the first challenge. Everything before was just messing around with the setup and basics for Lisp and CRAM. You've learned how to write Lisp code, define functions and parameters, create and visualize poses, and how to use one of the key components of CRAM: Designators. Now we will utilize all of that by making the robot pick and place objects from one place to another.

To use CRAM in this lesson we need to clean up a bit. Close the last Bullet World window. It is bound to the kernel of lesson 2, so it can't communicate with it when we work here in lesson 3. Then load the package, initialize the simulator again and define the tutorial package for your own code.

In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)

In [ ]:
(roslisp-utilities:startup-ros)

In [ ]:
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)
  (:use :common-lisp :cram-prolog :desig :exe))

In [ ]:
(in-package :btw-tut)

We also want to re-use the `make-pose` function from lesson 2 again, so compile it here.

In [ ]:
(defun make-pose (frame point euler)
    (declare (type string frame)
             (type list point euler))
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler."
    (if (and (eq (length point) 3) (eq (length euler) 3))
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        (format T "point ~a or euler ~a is not of length 3" point euler)))

## Spawning Objects and simulating

We've spawned objects before, in lesson 2. This is how we did it.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))

Notice that the `spawn-object` function has more optional slots to fill.

In [ ]:
(describe 'btr-utils:spawn-object)

There are the argments name, type, pose, color, mass and world. Only the first two are mandadory, the other ones use default values if they are not set explicitly. We recommend to set the `pose` argument though, otherwise it will spawn in the coordinate (2 0 0). Mass doesn't really have any effect in bullet, unless it's 0, then it won't fall down.

You can set the color though.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 1.82) '(0 0 0)) :color '(1 0 0))

Kill an object like this

In [ ]:
(btr-utils:kill-object 'bottle-1)
;; or (btr-utils:kill-all-objects) if you want to get rid of them all

To find all possible `obj-type`, the parameter `btr::*mesh-files*` contains the mesh files as key-value pairs between keyword and the location of their mesh-file. We can map over that list and return their keys. 

In [ ]:
(mapcar #'first btr::*mesh-files*)

After you've spawned an object, you can simulate the world to let it fall. Spawn your object higher above (increase the z-coordinate) and let it fall.

In [ ]:
(btr:simulate btr:*current-bullet-world* 10) ;; in seconds

If you want, you can do this in real time by simulating step by step. 

In [ ]:
(dotimes (i 10)
    (sleep 0.1)
    (btr:simulate btr:*current-bullet-world* 0.1))

And to make it look fancy, define a function for that.

In [ ]:
(defun simulate (&key (duration 5.0)    ;; in seconds 
                      (framerate 20.0)) ;; in Hz
    (let ((sleep-interval (/ 1.0 framerate))
          (times (floor (* duration framerate))))
         (dotimes (_ times)
             (sleep (/ sleep-interval 2))
             (btr:simulate btr:*current-bullet-world* (/ sleep-interval 2)))))

In [ ]:
(simulate)

In [ ]:
(simulate :duration 2)

In [ ]:
(simulate :framerate 5)

In [ ]:
(simulate :duration 1 :framerate 10)

I know what you're thinking. Here's how to make it rain.

In [ ]:
(let ((number-of-objects 20)
      (x-y-scatter-factor 0.8)
      (z-scatter-factor 0.5)
      (z-offset 2.0))
     (loop for n in (alexandria:iota number-of-objects)
           for object-name = (make-symbol (concatenate 'string "OBJECT-" (format NIL "~a" n)))
           for object-type = (nth (random (length btr::*mesh-files*)) (mapcar #'first btr::*mesh-files*))
           for x = (* (- (random 1.0) 0.5) x-y-scatter-factor)
           for y = (* (- (random 1.0) 0.5) x-y-scatter-factor)
           for z = (+ (* (random 1.0) z-scatter-factor) z-offset)
           do (btr-utils:spawn-object object-name object-type :pose `((,x ,y ,z) (0 0 0 1)))))

In [ ]:
(simulate)

In [ ]:
(btr-utils:kill-all-objects)

In [ ]:
(roslisp-utilities:startup-ros) ;; you're welcome

## Moving Around

Moving the robot is done by performing an action of type `going`. You can move the robot off the ground, but that seems unreasonable. The `going` action is very forgiving in that regard. Feel free to move the robot around. It will throw a `low-level-failure` if your taget pose would make the robot collide with the environment.

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal (make-pose "map" '(0.0 1.0 0.0) '(0.0 0.0 (* pi 0.0)))))
    (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

The robot will operate on the table to the right. Here is his base position.

In [ ]:
(defparameter *base-pose-near-table*
  (make-pose "map" '(-2.2d0 -0.20d0 0.0d0) '(0.0d0 0.0d0 (* pi -0.5))))

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
     (perform (an action
                  (type going)
                  (target (a location 
                             (pose ?navigation-goal)))))))

Besides moving, the robots motors can be also moved. To move the torso up and down, we can use a motion instead of an action. Motions are atomic actions without much logic. They are the most low-level plans, communicating directly with the process-modules.

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (a motion
             (type moving-torso)
             (joint-angle 0.3))))

Also the arms can be moved into standard positions, e.g. to move safely or to keep them out of the cameras field of view.

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (an action
              (type positioning-arm)
              (left-configuration tucked)
              (right-configuration tucked))))

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (an action
              (type positioning-arm)
              (left-configuration park)
              (right-configuration park))))

Altogether the robot can move its base, torso and arms all at once. This is done by wrapping all three action into `cpl:par`, which is a mechanism of CRAM that allows actions to be executed in parallel. In certain situations it is safer to execute actions explicitly in sequence, this can be expressed with `cpl:seq`. The following block executes the previous action in parallel.

In [ ]:
(urdf-proj:with-simulated-robot
 (let ((?navigation-goal *base-pose-near-table*))
      (cpl:par
       ;; Move the robot near the table.
       (perform (an action
                    (type going)
                    (target (a location
                               (pose ?navigation-goal)))))
       ;; Increasing the height of the torso by setting the joint angle to 0.3 meters
       (perform (a motion
                   (type moving-torso)
                   (joint-angle 0.3)))
       ;; This is a shorthand for bringing both arms into the park configuration
       (perform (an action
                    (type parking-arms)))))) 

## Perceiving Objects
Perceiving is also done via action designators. The action of type `detecting` is the one from lesson 2. Let's get our bottle back first.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))

To perceive something, the robot needs to look at the point of interest. We define ourself a pose. Notice that the coordinate frame has `base_footprint` as reference, which is the frame of the robots base. Like that, the pose is relative to the position of the robot, instead of the world.

In [ ]:
(defparameter *downward-look-coordinate*
 (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))

To look at that pose, we use the `looking` action.

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?looking-direction *downward-look-coordinate*))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction)))))))

Now that we roughly look down onto the table where the bottle stands, we can perceive the bottle with the `detecting` action. 

In [ ]:
(urdf-proj:with-simulated-robot
    (perform (an action
                 (type detecting)
                 (object (an object
                             (type bottle))))))

The object is found, let's pick it up.

## Picking up objects
The action we need is `picking-up`. One of its parameters is the perceived bottle. Make sure that the robot is still facing the bottle.

In [ ]:
(urdf-proj:with-simulated-robot 
 (let ((?perceived-bottle (perform (an action
                                       (type detecting)
                                       (object (an object
                                                   (type bottle)))))))
      (perform (an action
                   (type picking-up)
                   (arm (right))
                   (object ?perceived-bottle)))))

And afterwards the robot needs to go back to its parking position.

In [ ]:
(urdf-proj:with-simulated-robot 
 (exe:perform
  (desig:an action
            (type parking-arms))))

## Placing Objects

To place the object currently grasped, we need a pose where the robot should stand, and a target location for the object.

In [ ]:
(defparameter *object-delivery-pose*
  (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
 
(defparameter *base-pose-near-kitchen-island*
  (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

While the bottle is in the gripper, move the robot over to the pose in front of the kitchen island.

In [ ]:
(urdf-proj:with-simulated-robot
 (let ((?delivery-base-pose *base-pose-near-kitchen-island*))
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?delivery-base-pose)))))))

Then call the `placing` action. 

In [ ]:
(urdf-proj:with-simulated-robot
 (let ((?object-delivery-pose *object-delivery-pose*))
      (perform (an action
                   (type placing)
                   (target (a location
                              (pose ?object-delivery-pose)))))))

Notice that the action doesn't need to be told which arm to use, or what kind of object he is holding. These are optional parameters that can be inferred from the current world state. The robot knows what object he is holding, and also in which hand he does.

When we bring it all together, pick and place, the whole procedure can look like this.

In [ ]:
;; Reset the world
(demo::initialize)

;; Have all parameters we need in one place.
(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

;; Spawn the bottle
(btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)

(urdf-proj:with-simulated-robot
 ;; Go to the table where the bottle stands
 (let ((?navigation-goal *base-pose-near-table*))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 ;; Look down at the bottle
 (let ((?looking-direction *downward-look-coordinate*))
      (perform (an action
                   (type looking)
                   (target (a location 
                              (pose ?looking-direction))))))
 ;; Detect the bottle and keep the object designator in ?perceived-bottle
 ;; so we can use it later
 (let ((?nav-goal *base-pose-near-kitchen-island*)
       (?drop-pose *object-delivery-pose*)
       (?perceived-bottle (perform (an action
                                       (type detecting)
                                       (object (an object
                                                   (type bottle)))))))
      ;; Pick up the detected bottle
      (perform (an action
                   (type picking-up)
                   (arm (right))
                   (object ?perceived-bottle)))
      ;; Park arms
      (perform (an action
                   (type parking-arms)))
      ;; Move the torso up so the robot can reach the higher surface better
      ;; without crashing his arms into it.
      (perform (a motion 
                  (type moving-torso)
                  (joint-angle 0.3)))
      ;; Go to the target delivery location
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?nav-goal)))))
      ;; Place down the bottle
      (perform (an action
                   (type placing)
                   (arm (right))
                   ;; here we explicitly specify to put down the bottle
                   ;; it's not necessary, but you can use this as a reference for the upcoming task
                   (object ?perceived-bottle) 
                   (target (a location
                              (pose ?drop-pose)))))))

## The stage is yours

Now you know how to pick and place objects. Are you ready to write your own plan?

### Task: Switch the position of 2 objects

Spawn two bottles on two different surfaces in the kitchen. The robot is able two carry two objects if you don't specify which arm to use in the `picking-up` action. Also `placing` doesn't need any arm specified, but you can specify the object designator, if you keep track of it, like in the complete pick-place plan above. Find suitable poses for the objects and the robot. Visualize, move, look and try things out. You can always reset the Bullet World with

In [ ]:
(roslisp-utilities:startup-ros)   ;; when Bullet hangs up, use this, it's the hardest reset except for restarting Docker
(demo::initialize)                ;; initializes the robot, environment and objects
(btr-utils:kill-all-objects)      ;; removes all objects
(btr-utils:kill-object 'bottle-1) ;; removes the 'bottle-1 object

In [ ]:
(let ((the-hacking "begin"))
     )